# Análisis componentes principales

* Estandarizar los datos (para cada una de las m observaciones)
* Obtener los vectores y valores propios a partir de la matriz de covarianzas o de correlaciones o incluso la técnica de singular vector decomposition.
* Ordenar los valores propios en orden descendente y quedarnos con los *p* que se correpondan a los *p* mayores y así disminuir el nº de vars. del dataset (p<m)
* Constrir la matriz de proyección W a partir de los p vectores propios.
* Transformar el dataset original X a través de W para así obtener datos en el subespacio dimensional de dimensión *p*, que será Y.

In [64]:
import pandas as pd

import chart_studio
import chart_studio.plotly as py
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler

from IPython.display import display, Math, Latex

import numpy as np

In [46]:
mainpath = "/Users/irene/Documents/GitHub/python-ml-course/datasets"  #Ruta ficheros
filename = "iris/iris.csv" #Fichero a abrir
fullpath = mainpath + "/" + filename #Ruta completa

df = pd.read_csv(fullpath)
df.head(3)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


In [43]:
#Lo guardamos en formato array, en lugar de dataset

X = df.iloc[:,0:4].values #Parámetros con los que trabajar
y = df.iloc[:,4].values #Species, esta columna no se puede transformar linealmente

# Análisis exploratorio de datos

Usamos la librería plotly para visualizar gráficamente cómo se distribuyen los datos en un histograma

In [53]:
traces = [] #array vacío
legend = {0:True, 1:True, 2:True, 3:True} #creamos un diccionario

colors = {'setosa': 'rgb(255,127,20)', #diccionario con colores para cada tipo
         'versicolor': 'rgb(31, 220, 120)',
         'virginica': 'rgb(44, 50, 180)'}

#para construir los colores
for col in range(4): #col es para colores
    for key in colors: #key es para cada tipo de flor
        traces.append(go.Histogram(x=X[y==key, col], opacity = 0.7, #construimos un histograma
            xaxis="x%s"%(col+1), marker=go.Marker(color=colors[key]),
            name = key, showlegend=legend[col]))
    legend = {0:False, 1:False, 2:False, 3:False}

data = go.Data(traces)

layout = go.Layout(barmode="overlay", #para pintar un diagrama de barras
    xaxis=go.XAxis(domain=[0,0.25], title="Long. Sépalos (cm)"), #eje x, dominio 1
    xaxis2=go.XAxis(domain=[0.3, 0.5], title = "Anch. Sépalos (cm)"), #eje x, dominio 2
    xaxis3=go.XAxis(domain = [0.55, 0.75], title = "Long. Pétalos (cm)"), #eje x, dominio 3
    xaxis4=go.XAxis(domain=[0.8,1.0], title = "Anch. Pétalos (cm)"), #eje x, dominio 4
    yaxis=go.YAxis(title="Número de ejemplares"), #eje y
    title="Distribución de los rasgos de las diferentes flores Iris")

fig = go.Figure(data = data, layout = layout) #creamos la figura
fig.show()

# Paso 1. Estandarización / Normalización de datos

Todos media 0

In [57]:
from sklearn.preprocessing import StandardScaler

In [58]:
X_std = StandardScaler().fit_transform(X)

# Paso 2. Calculamos la descomposición de valores y vectores propios
### a) Usando la Matriz de Covarianzas

A partir de la matriz de covarianzas, se extraen los vectores propios, mientras que los valores propios determinan la magnitud, explican la varianza.

La matriz de covarianza es una matriz nxn que analiza la covarianza entre 2 variables.

In [65]:
#cálculo de la covarianza
display(Math(r'\sigma_{jk} = \frac{1}{n-1}\sum_{i=1}^m (x_{ij} - \overline{x_j})(x_{ik} - \overline{x_k})')) 

<IPython.core.display.Math object>

In [66]:
#fórmula comprimida de la covarianza
display(Math(r'\Sigma = \frac{1}{n-1}((X-\overline{x})^T(X-\overline{x}))'))

<IPython.core.display.Math object>

In [67]:
display(Math(r'\overline{x} = \sum_{i=1}^n x_i\in \mathbb R^m'))

<IPython.core.display.Math object>

In [68]:
# Implementación manual

mean_vect = np.mean(X_std, axis=0) #promedio de las 4 vars
mean_vect #vector de medias

array([-4.73695157e-16, -7.81597009e-16, -4.26325641e-16, -4.73695157e-16])

In [69]:
#matriz de covarianza

cov_matrix = (X_std - mean_vect).T.dot((X_std - mean_vect))/(X_std.shape[0]-1) #fórmula comprimida
print("La matriz de covarianzas es \n%s"%cov_matrix) #la diagonal siempre tiene 1's

La matriz de covarianzas es 
[[ 1.00671141 -0.11835884  0.87760447  0.82343066]
 [-0.11835884  1.00671141 -0.43131554 -0.36858315]
 [ 0.87760447 -0.43131554  1.00671141  0.96932762]
 [ 0.82343066 -0.36858315  0.96932762  1.00671141]]


In [72]:
#alternativa automática con numpy

np.cov(X_std.T)

array([[ 1.00671141, -0.11835884,  0.87760447,  0.82343066],
       [-0.11835884,  1.00671141, -0.43131554, -0.36858315],
       [ 0.87760447, -0.43131554,  1.00671141,  0.96932762],
       [ 0.82343066, -0.36858315,  0.96932762,  1.00671141]])

In [74]:
#descomposición de la matriz de covarianza en valor y vector

eig_vals, eig_vectors = np.linalg.eig(cov_matrix) #eig = propios
print("Valores propios \n%s"%eig_vals) #aparecen ya ordenados
print("Vectores propios \n%s"%eig_vectors)

Valores propios 
[2.93808505 0.9201649  0.14774182 0.02085386]
Vectores propios 
[[ 0.52106591 -0.37741762 -0.71956635  0.26128628]
 [-0.26934744 -0.92329566  0.24438178 -0.12350962]
 [ 0.5804131  -0.02449161  0.14212637 -0.80144925]
 [ 0.56485654 -0.06694199  0.63427274  0.52359713]]


### b) Usando la Matriz de Correlaciones

Generalmente usado en el campo de las finanzas. Es una versión normalizada de la matriz de covarianzas.
Es decir, la descomposición de una matriz de covarianzas es la misma que la de la matriz de correlaciones estandarizada o no.

In [78]:
corr_matrix = np.corrcoef(X_std.T)
corr_matrix #ahora está totalmente normalizada, diagonal con 1's exactos

array([[ 1.        , -0.11756978,  0.87175378,  0.81794113],
       [-0.11756978,  1.        , -0.4284401 , -0.36612593],
       [ 0.87175378, -0.4284401 ,  1.        ,  0.96286543],
       [ 0.81794113, -0.36612593,  0.96286543,  1.        ]])

In [80]:
eig_vals_corr, eig_vectors_corr = np.linalg.eig(corr_matrix) #difieren un pelín de lo anterior
print("Valores propios \n%s"%eig_vals_corr)
print("Vectores propios \n%s"%eig_vectors_corr)

Valores propios 
[2.91849782 0.91403047 0.14675688 0.02071484]
Vectores propios 
[[ 0.52106591 -0.37741762 -0.71956635  0.26128628]
 [-0.26934744 -0.92329566  0.24438178 -0.12350962]
 [ 0.5804131  -0.02449161  0.14212637 -0.80144925]
 [ 0.56485654 -0.06694199  0.63427274  0.52359713]]


In [81]:
corr_matrix = np.corrcoef(X.T)
corr_matrix

array([[ 1.        , -0.11756978,  0.87175378,  0.81794113],
       [-0.11756978,  1.        , -0.4284401 , -0.36612593],
       [ 0.87175378, -0.4284401 ,  1.        ,  0.96286543],
       [ 0.81794113, -0.36612593,  0.96286543,  1.        ]])

### c) Singular Value Decomposition

Última técnica. Mejora mucho la eficacia computacional.

In [86]:
u,s,v = np.linalg.svd(X_std.T)
u #matriz de vectores propios

array([[-0.52106591, -0.37741762,  0.71956635,  0.26128628],
       [ 0.26934744, -0.92329566, -0.24438178, -0.12350962],
       [-0.5804131 , -0.02449161, -0.14212637, -0.80144925],
       [-0.56485654, -0.06694199, -0.63427274,  0.52359713]])

In [87]:
s #no se parece a los valores propios

array([20.92306556, 11.7091661 ,  4.69185798,  1.76273239])

In [85]:
v

array([[ 1.08239531e-01,  9.94577561e-02,  1.12996303e-01, ...,
        -7.27030413e-02, -6.56112167e-02, -4.59137323e-02],
       [-4.09957970e-02,  5.75731483e-02,  2.92000319e-02, ...,
        -2.29793601e-02, -8.63643414e-02,  2.07800179e-03],
       [ 2.72186462e-02,  5.00034005e-02, -9.42089147e-03, ...,
        -3.84023516e-02, -1.98939364e-01, -1.12588405e-01],
       ...,
       [ 5.43380310e-02,  5.12936114e-03,  2.75184277e-02, ...,
         9.89532683e-01, -1.41206665e-02, -8.30595907e-04],
       [ 1.96438400e-03,  8.48544595e-02,  1.78604309e-01, ...,
        -1.25488246e-02,  9.52049996e-01, -2.19201906e-02],
       [ 2.46978090e-03,  5.83496936e-03,  1.49419118e-01, ...,
        -7.17729676e-04, -2.32048811e-02,  9.77300244e-01]])

# Paso 3. Las componentes principales

Una vez descompuesta la matriz de covarianza, elegimos los componentes principales.
La longitud de todos los vectores propios nos debería dar 1, lo comprobamos primero:

In [88]:
for ev in eig_vectors:
    print("La longitud del VP es: %s"%np.linalg.norm(ev))

La longitud del VP es: 0.9999999999999997
La longitud del VP es: 1.0000000000000002
La longitud del VP es: 1.0
La longitud del VP es: 0.9999999999999997


In [89]:
eigen_pairs = [(np.abs(eig_vals[i]), eig_vectors[:,i]) for i in range(len(eig_vals))]
eigen_pairs

[(2.938085050199993,
  array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654])),
 (0.9201649041624873,
  array([-0.37741762, -0.92329566, -0.02449161, -0.06694199])),
 (0.1477418210449481,
  array([-0.71956635,  0.24438178,  0.14212637,  0.63427274])),
 (0.020853862176462803,
  array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))]

Ordenamos los vectores propios con valor propio de mayor a menor

In [91]:
eigen_pairs.sort()
eigen_pairs.reverse()
eigen_pairs

[(2.938085050199993,
  array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654])),
 (0.9201649041624873,
  array([-0.37741762, -0.92329566, -0.02449161, -0.06694199])),
 (0.1477418210449481,
  array([-0.71956635,  0.24438178,  0.14212637,  0.63427274])),
 (0.020853862176462803,
  array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))]

In [92]:
print("Valores propios en orden descendente:")
for ep in eigen_pairs:
    print(ep[0])

Valores propios en orden descendente:
2.938085050199993
0.9201649041624873
0.1477418210449481
0.020853862176462803


In [93]:
total_sum = sum(eig_vals)
var_exp = [(i/total_sum)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

In [95]:
plot1 = go.Bar(x=[f"CP {i}" for i in range(1,5)], y=var_exp, showlegend= True)
plot2 = go.Scatter(x=[f"CP {i}" for i in range(1,5)], y=cum_var_exp, showlegend= True)

data = [plot1,plot2]

layout = go.Layout(xaxis= {"title": "Componentes principales"},
                  yaxis ={"title": "Porcentaje de varianza explicada"},
                  title = "Porcentaje de variabilidad explicada por cada componente principal")

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)
fig.show()

PlotlyRequestError: User cannot save file.
Account limit reached: Your account is limited to creating 100 charts. To continue, you can override or delete existing charts or you can upgrade your account at: https://plotly.com/products/cloud

In [96]:
W = np.hstack((eigen_pairs[0][1].reshape(4,1), 
               eigen_pairs[1][1].reshape(4,1)))
W

array([[ 0.52106591, -0.37741762],
       [-0.26934744, -0.92329566],
       [ 0.5804131 , -0.02449161],
       [ 0.56485654, -0.06694199]])

In [97]:
X[0]

array([5.1, 3.5, 1.4, 0.2])

## Paso 4. Proyectando las variables en el nuevo subespacio vectorial

In [98]:
display(Math(r'Y = X \cdot W, X \in M(\mathbb R)_{150, 4}, W \in M(\mathbb R)_{4,2}, Y \in M(\mathbb R)_{150, 2}'))

<IPython.core.display.Math object>

In [99]:
Y = X_std.dot(W)
Y

array([[-2.26470281, -0.4800266 ],
       [-2.08096115,  0.67413356],
       [-2.36422905,  0.34190802],
       [-2.29938422,  0.59739451],
       [-2.38984217, -0.64683538],
       [-2.07563095, -1.48917752],
       [-2.44402884, -0.0476442 ],
       [-2.23284716, -0.22314807],
       [-2.33464048,  1.11532768],
       [-2.18432817,  0.46901356],
       [-2.1663101 , -1.04369065],
       [-2.32613087, -0.13307834],
       [-2.2184509 ,  0.72867617],
       [-2.6331007 ,  0.96150673],
       [-2.1987406 , -1.86005711],
       [-2.26221453, -2.68628449],
       [-2.2075877 , -1.48360936],
       [-2.19034951, -0.48883832],
       [-1.898572  , -1.40501879],
       [-2.34336905, -1.12784938],
       [-1.914323  , -0.40885571],
       [-2.20701284, -0.92412143],
       [-2.7743447 , -0.45834367],
       [-1.81866953, -0.08555853],
       [-2.22716331, -0.13725446],
       [-1.95184633,  0.62561859],
       [-2.05115137, -0.24216355],
       [-2.16857717, -0.52714953],
       [-2.13956345,

In [100]:
results = []
for name in ('setosa', 'versicolor', 'virginica'):
    result = go.Scatter(x= Y[y==name,0], y =Y[y==name, 1],
                       mode = "markers", name=name,
    marker= { "size": 12, "line" : { "color" : 'rgba(220,220,220,0.15)', "width":0.5},
           "opacity": 0.8})
    results.append(result)
    
layout = go.Layout(showlegend = True, 
                   scene ={ "xaxis" :{"title": "Componente Principal 1"},
                            "yaxis" : {"title": "Componente Principal 2"}},
                  xaxis ={ "zerolinecolor": "gray"},
                  yaxis={ "zerolinecolor": "gray"})
fig = go.Figure(data=results,layout=layout)
py.iplot(fig)
#fig.show()

PlotlyRequestError: User cannot save file.
Account limit reached: Your account is limited to creating 100 charts. To continue, you can override or delete existing charts or you can upgrade your account at: https://plotly.com/products/cloud